In [1]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
# import bff
# import pingouin as pg
#import researchpy
# Plotting packages
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

In [2]:
df = pd.read_excel("data63.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,Edu,Income,Indirect_SES,MacArthur_SES,PerceivedSciConsensu_1,Intro_Timer,condition_time_total,WEPTcc,BELIEFcc,POLICYcc
0,R_1r7Yr7R1xMimIvj,usa,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,98.0,77.0,86.0,93.0,71.0,...,3.0,5.0,"2,4,5",7.0,80.0,6.056,95.501,1,88.50,89.000000
1,R_2ATgV5VxewrTRiI,usa,1,Control,PolicySupport|BeliefinAnthrCC|Socialmediashari...,49.0,50.0,51.0,51.0,1.0,...,3.0,4.0,"1,2,3,4,5,6,7",5.0,51.0,16.284,200.742,0,50.25,66.222222
2,R_3ENlePBlLyYjExr,usa,9,LetterFutureGen,SocialmediasharingPipedText|PolicySupport|Beli...,87.0,82.0,84.0,83.0,84.0,...,3.0,5.0,"1,2,3,6,7",8.0,73.0,5.701,218.442,1,84.00,84.222222
3,R_2c0HQgvYzas6mus,usa,4,SciConsens,PolicySupport|SocialmediasharingPipedText|Beli...,1.0,1.0,0.0,0.0,1.0,...,3.0,5.0,"1,2,3,4,5,6,7",4.0,1.0,13.282,12.702,0,0.50,1.222222
4,R_00siryCwgdMus6J,usa,8,PluralIgnorance,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,56.0,59.0,61.0,54.0,79.0,...,3.0,5.0,"1,3,5,6,7",7.0,65.0,8.474,19.315,0,57.50,80.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,3.0,2.0,"1,2,3,4,5,6,7",7.0,60.0,27.732,873.538,0,80.75,82.500000
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,3.0,2.0,"1,2,4,6,7",3.0,74.0,49.502,0.000,0,96.50,86.444444
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,3.0,1.0,"2,4,5,6,7",6.0,100.0,7.920,89.072,7,100.00,83.333333
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,3.0,NaN,"1,2,3,4,5,6,7",7.0,100.0,40.222,398.193,8,64.75,66.666667


In [3]:
# Per-country median split
df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

df_Lib = df.query('Ideology == "Liberal"')
df_Con = df.query('Ideology == "Conservative"')

df_Lib_b = pd.melt(
    df_Lib.loc[:, ['ResponseId', "Country", "condName", "Ideology","ide","cond", "WEPTcc", "Belief1","Belief2","Belief3","Belief4"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Belief"
)

df_Con_b = pd.melt(
    df_Con.loc[:, ['ResponseId', "Country", "condName", "Ideology","ide", "cond", "WEPTcc", "Belief1","Belief2","Belief3","Belief4"]],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","ide", "cond", "WEPTcc"],
    var_name="Item",
    value_name="Belief"
)

df_Lib_b = df_Lib_b.dropna()
df_Con_b = df_Con_b.dropna()

df_Con_b['condName'] = df_Con_b['condName'].replace(['Control'], 'aControl')
df_Lib_b['condName'] = df_Lib_b['condName'].replace(['Control'], 'aControl')

/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [4]:
df_Lib = df.query('Ideology == "Liberal"')
df_Con = df.query('Ideology == "Conservative"')

df_Lib_p = pd.melt(
    df_Lib.loc[:, ['ResponseId', "Country", "condName","Ideology","cond","ide","WEPTcc", 'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', "Country", "condName", "Ideology","cond","ide", "WEPTcc"],
    var_name="Item",
    value_name="Policy"
)

df_Con_p = pd.melt(
    df_Con.loc[:, ['ResponseId', "Country", "condName","Ideology","cond","ide","WEPTcc", 'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', "Country", "condName",  "Ideology","cond", "ide", "WEPTcc"],
    var_name="Item",
    value_name="Policy"
)

df_Lib_p = df_Lib_p.dropna()
df_Con_p = df_Con_p.dropna()

df_Lib_p['condName'] = df_Lib_p['condName'].replace(['Control'], 'aControl')
df_Con_p['condName'] = df_Con_p['condName'].replace(['Control'], 'aControl')


In [11]:
df_Lib_w = df.query('Ideology == "Liberal"')
df_Con_w = df.query('Ideology == "Conservative"')

#df_Lib_w['condName'] = df_Lib_w['condName'].replace(['Control'], 'aControl')
#df_Con_w['condName'] = df_Con_w['condName'].replace(['Control'], 'aControl')
df_Lib_w['condName'] = df_Lib_w['condName'].replace(['Control'], 'aControl')
df_Con_w['condName'] = df_Con_w['condName'].replace(['Control'], 'aControl')

# Analyses

In [12]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython
# Load the R library lmerTest
%R library(lmerTest)
%R library(ordinal)
# Send R the dataset
%Rpush df_Lib_b
%Rpush df_Con_b

%Rpush df_Lib_p
%Rpush df_Con_p

%Rpush df_Lib_w
%Rpush df_Con_w

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Table S1

In [13]:
%%R

M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Lib_b)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Lib_b

REML criterion at convergence: 850526.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.6847 -0.2146  0.0833  0.2689  6.3660 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 423.6672 20.5832 
 Country    (Intercept)  38.8745  6.2349 
 Item       (Intercept)   0.5095  0.7138 
 Residual               130.8334 11.4382 
Number of obs: 101581, groups:  ResponseId, 25416; Country, 60; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 81.3157     1.0012    74.6828  81.220  < 2e-16 ***
condNameBindingMoral         1.3922     0.6524 25341.8982   2.134  0.03287 *  
condNameCollectAction        1.8019     0.6527 25336.9555   2.761  0.00577 ** 
condNameDynamicNorm          0.5538     0.6495 

# Table S2

In [16]:
%%R

M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Con_b)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Con_b

REML criterion at convergence: 833422.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1948 -0.3224  0.0856  0.3314  5.9335 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 544.7323 23.3395 
 Country    (Intercept)  52.8422  7.2693 
 Item       (Intercept)   0.1732  0.4162 
 Residual               151.7459 12.3185 
Number of obs: 97530, groups:  ResponseId, 24408; Country, 60; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 78.4371     1.1059    99.8881  70.928  < 2e-16 ***
condNameBindingMoral         0.9237     0.7498 24332.5604   1.232  0.21799    
condNameCollectAction        2.5750     0.7463 24336.0544   3.451  0.00056 ***
condNameDynamicNorm          0.3626     0.7474 2

# Table S3

In [17]:
%%R

M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Lib_p)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Lib_p

REML criterion at convergence: 2014942

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.7963 -0.5052  0.0722  0.6285  4.9034 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 285.62   16.900  
 Country    (Intercept)  25.12    5.012  
 Item       (Intercept) 149.90   12.243  
 Residual               399.50   19.988  
Number of obs: 222531, groups:  ResponseId, 25318; Country, 60; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 70.8429     4.1521     8.5626  17.062 6.57e-08 ***
condNameBindingMoral         0.6627     0.5573 25167.5277   1.189 0.234418    
condNameCollectAction        3.0100     0.5579 25156.5662   5.395 6.92e-08 ***
condNameDynamicNorm          0.7744     0.5546 2

# Table S4

In [18]:
%%R

M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_Con_p)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_Con_p

REML criterion at convergence: 1942246

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8637 -0.5497  0.0535  0.6371  4.4416 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 311.32   17.644  
 Country    (Intercept)  41.76    6.462  
 Item       (Intercept) 140.92   11.871  
 Residual               416.38   20.405  
Number of obs: 213415, groups:  ResponseId, 24327; Country, 60; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 70.0668     4.0668     8.9281  17.229 3.71e-08 ***
condNameBindingMoral         0.5455     0.5899 24218.8221   0.925  0.35511    
condNameCollectAction        2.6460     0.5866 24212.5560   4.511 6.49e-06 ***
condNameDynamicNorm          0.6324     0.5878 2

# Table S5


In [19]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName + (1|Country), data = df_Lib_w, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_Lib_w

 link  threshold   nobs  logLik    AIC      niter      max.grad cond.H 
 logit equidistant 26544 -43416.36 86860.72 1737(6954) 2.84e-02 5.9e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2325   0.4821  
Number of groups:  Country 60 

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral      0.101182   0.057154   1.770 0.076672 .  
condNameCollectAction    -0.064592   0.056580  -1.142 0.253622    
condNameDynamicNorm       0.098280   0.057003   1.724 0.084685 .  
condNameFutureSelfCont   -0.006718   0.060239  -0.112 0.911202    
condNameLetterFutureGen  -0.172546   0.060047  -2.874 0.004059 ** 
condNameNegativeEmotions -0.248452   0.055166  -4.504 6.68e-06 ***
condNamePluralIgnorance  -0.033591   0.056432  -0.595 0.551685    
condNamePsychDistance    -0.211661   0.

# Table S6

In [20]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName + (1|Country), data = df_Con_w, threshold = "equidistant")
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_Con_w

 link  threshold   nobs  logLik    AIC      niter      max.grad cond.H 
 logit equidistant 24642 -39704.41 79436.82 1768(7083) 6.33e-02 6.3e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2776   0.5269  
Number of groups:  Country 60 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral     -0.02927    0.05929  -0.494  0.62154    
condNameCollectAction    -0.14608    0.05863  -2.491  0.01272 *  
condNameDynamicNorm      -0.10119    0.05854  -1.729  0.08386 .  
condNameFutureSelfCont   -0.21225    0.06160  -3.445  0.00057 ***
condNameLetterFutureGen  -0.32122    0.06185  -5.194 2.06e-07 ***
condNameNegativeEmotions -0.35380    0.05919  -5.978 2.27e-09 ***
condNamePluralIgnorance  -0.12366    0.05852  -2.113  0.03460 *  
condNamePsychDistance    -0.34813    0.05946  -